# Test Distributional Shift
This notebook tests distributional shift for the dataset of Reddit posts.

## Imports

In [1]:
%%capture
!pip install transformers

In [2]:
%%capture
!pip install datasets

In [3]:
%%capture
!pip install nltk

In [4]:
%%capture
import nltk
nltk.download('punkt')

In [5]:
from transformers import BertTokenizer, BertModel, AlbertTokenizer, AlbertModel, DistilBertTokenizer, DistilBertModel
import torch
from datasets import load_dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from math import ceil

from nltk.tokenize import sent_tokenize, word_tokenize
import random
import numpy as np

## Loading and Preparing Texts

In [6]:
def remove_sentence(t, k=1):
    sentences = random.sample(sent_tokenize(t), k)
    text = t
    for s in sentences:
        text = text.replace(s, '')
        assert len(t) != len(text)
    return text, len(word_tokenize(s))

def remove_word(t, k=1):
    text = t
    for _ in range(k):
        words = word_tokenize(text)
        if not len([i for i,w in enumerate(words) if w.isalnum()]):
            break
        word_idx = random.choice([i for i,w in enumerate(words) if w.isalnum()])
        del words[word_idx]
        text = ''.join([(' ' if w.isalnum() else '')+w for w in words]).strip(' ')
    return text

In [7]:
dataset = load_dataset('squadshifts', 'reddit')['test']
print('Average number of sentences = {}'.format(np.mean([len(sent_tokenize(sample['context'])) for sample in dataset ])))

Dataset squad_shifts downloaded and prepared to /root/.cache/huggingface/datasets/squad_shifts/reddit/1.0.0/f6c7b6f10e62b342754f88631c92624a2033652e3a3e129b8d979726dec04039. Subsequent calls will reuse this data.
Average number of sentences = 8.860246863205141


In [ ]:
N_WORD = 5
texts = []
dataset = load_dataset('squadshifts', 'reddit')['test']
for d in tqdm(dataset):
    if d['context'] not in texts:
        texts.append(d['context'])
random.shuffle(texts)        
n_samples = len(texts)
original = texts#random.sample(texts, n_samples)#texts[:n_samples]
word_removed = []
sentence_removed = []
for t in tqdm(original):
    sr, nw = remove_sentence(t)
    wr = remove_word(t, k=N_WORD)
    word_removed.append(wr)
    sentence_removed.append(sr)

Reusing dataset squad_shifts (/root/.cache/huggingface/datasets/squad_shifts/reddit/1.0.0/f6c7b6f10e62b342754f88631c92624a2033652e3a3e129b8d979726dec04039)
100%|██████████| 2033/2033 [00:11<00:00, 173.36it/s]


In [ ]:
text_data = original+word_removed+sentence_removed
classes = [
    (n_samples, 'Original', 'tab:blue'),
    (n_samples, 'Word', 'tab:green'),
    (n_samples, 'Sentence', 'tab:red')
]

## Compare Distributional Shift
For different Classifiers (BERT, ALBERT, DistilBERT)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
def get_accuracy_rf(x_original, x_new):
    X = torch.cat([x_original, x_new], dim=0).cpu().numpy()
    y = np.array([0]*n_samples+[1]*n_samples)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    oob_score = 0
    best_clf = None
    for depth in [2, 5, 7, 10, 15, 20]:
        clf = RandomForestClassifier(max_depth=depth, oob_score=True)
        clf.fit(X_train, y_train)
        print('Depth {}, OOB score {}'.format(depth, clf.oob_score_))
        if clf.oob_score_>oob_score:
            oob_score = clf.oob_score_
            best_clf = clf
    print('--> Depth={}'.format(best_clf.max_depth))
    print('-'*20)
    train_acc = best_clf.score(X_train, y_train)
    test_acc = best_clf.score(X_test, y_test)
    print(X_train.shape)
    print('Train : {}({})'.format(train_acc, train_acc*(1-train_acc)/X_train.shape[0]))
    print('Test : {}({})'.format(test_acc, test_acc*(1-test_acc)/X_test.shape[0]))
    print('-'*20)

In [ ]:
def process_texts(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    return model(**(inputs.to(device)))[1].detach()

def batch(l, size):
    for i in range(0, len(l), size):
        yield l[i:i+size]

In [ ]:
def models_generator():
    model_classes = [
        BertModel,
        AlbertModel
    ]
    tokenizer_classes = [
        BertTokenizer,
        AlbertTokenizer
    ]
    model_names = [
        'bert-base-uncased',
        'albert-base-v2'
    ]
    for model_name, tokenizer_class, model_class in zip(model_names, tokenizer_classes, model_classes):
        tokenizer = tokenizer_class.from_pretrained(model_name)
        model = model_class.from_pretrained(model_name).to(device)
        yield model, tokenizer, model_name.split('-')[0].upper()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
for model, tokenizer, name in models_generator():
    print('Model: {}'.format(name))
    results = []
    for b in batch(text_data, 5):
        results.append(process_texts(b, model, tokenizer))
    embeddings = torch.cat(results, 0)
    e_original = embeddings[:n_samples, :]
    e_word = embeddings[n_samples:2*n_samples, :]
    e_sentence = embeddings[2*n_samples:, :]
    print('Sentence vs Original')
    get_accuracy_rf(e_original, e_sentence)
    print('Word vs Original')
    get_accuracy_rf(e_original, e_word)
    print('\n\n')

Model: BERT
Sentence vs Original
Depth 2, OOB score 0.538537225319777
Depth 5, OOB score 0.5119711380780584
Depth 7, OOB score 0.4801574286651361
Depth 10, OOB score 0.43719252213840604
Depth 15, OOB score 0.3879960642833716
Depth 20, OOB score 0.38832404066907183
--> Depth=2
--------------------
(3049, 768)
Train : 0.5769104624467039(8.00540442001424e-05)
Test : 0.5290068829891839(0.0002449937077082122)
--------------------
Word vs Original
Depth 2, OOB score 0.6388979993440472
Depth 5, OOB score 0.6559527714004592
Depth 7, OOB score 0.6474253853722532
Depth 10, OOB score 0.6280747786159396
Depth 15, OOB score 0.6254509675303378
Depth 20, OOB score 0.6110200065595277
--> Depth=5
--------------------
(3049, 768)
Train : 0.7569694981961299(6.0336725810701606e-05)
Test : 0.6411012782694199(0.0002262442765690617)
--------------------



Model: ALBERT
Sentence vs Original
Depth 2, OOB score 0.5529681862905871
Depth 5, OOB score 0.5234503115775664
Depth 7, OOB score 0.49491636602164646
Dept

In [ ]:
def process_texts_db(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    return model(**(inputs.to(device)))[0][:, 0, :].cpu().detach()

In [ ]:
model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
print('Model: DistilBert')
results = []
for b in batch(text_data, 2):
    results.append(process_texts_db(b, model, tokenizer))
embeddings = torch.cat(results, 0)
e_original = embeddings[:n_samples, :]
e_word = embeddings[n_samples:2*n_samples, :]
e_sentence = embeddings[2*n_samples:, :]
print('Sentence vs Original')
get_accuracy_rf(e_original, e_sentence)
print('Word vs Original')
get_accuracy_rf(e_original, e_word)
print('\n\n')

Model: DistilBert
Sentence vs Original
Depth 2, OOB score 0.4867169563791407
Depth 5, OOB score 0.405706789111184
Depth 7, OOB score 0.34863889799934406
Depth 10, OOB score 0.27189242374549033
Depth 15, OOB score 0.2249918005903575
Depth 20, OOB score 0.2151525090193506
--> Depth=2
--------------------
(3049, 768)
Train : 0.6300426369301411(7.644765909486898e-05)
Test : 0.5054080629301868(0.0002457922840268861)
--------------------
Word vs Original
Depth 2, OOB score 0.6769432600852738
Depth 5, OOB score 0.6598884880288619
Depth 7, OOB score 0.6179075106592326
Depth 10, OOB score 0.5572318793046901
Depth 15, OOB score 0.5182026894063627
Depth 20, OOB score 0.5204985241062644
--> Depth=2
--------------------
(3049, 768)
Train : 0.7284355526402099(6.487936972448734e-05)
Test : 0.6971484759095379(0.00020760322364458836)
--------------------



